# Hidden Number Problem

O problema do número escondido está relacionado com duas funções sobre elementos do corpo $Z_p$ e números naturais:
1. A primeira função, a **norma de um elemento em $Z_p$**, é definida da seguinte maneira:
    $\mid\mid X\mid\mid_p=$min$(x, p-x)$
1. A segunda função devolve os **$k$ bits mais significativos do seu argumento**, um número natural, módulo $p$: $msb_{k,p}(x)$

## Boneh & Venkatesan Algorithm

In [609]:
import numpy as np
import random as rand
import sage.crypto.lattice as lat
from sage.modules.free_module_integer import IntegerLattice

class BV:
    def __init__(self, p, k, l, s=4):
        self.p = p
        self.l = l
        self.lambd = 2 ** (k+1)
        self.x = [rand.randint(0, self.p-1) for i in range(0, self.l)]
        self.define_target(s)
        self.gen_lattice()

    def gen_lattice(self):
        ''' Calculate the lattice base
        '''
        b_1 = self.lambd * self.p * identity_matrix(ZZ, self.l)
        b_2 = self.lambd * matrix(self.x)
        self.G = block_matrix([[b_1, 0],[b_2, 1]])
        self.lat = IntegerLattice(self.G)
        self.lat_reduced = np.array(self.lat.reduced_basis)
    
    def msb(self, x):
        ''' Return an approximation of the k most significative bits of x
        '''
        valid_u = False
        while (not valid_u):
            u = rand.randint(0, self.p - 1)
            x_u = x - u
            norm_int = min(x_u, self.p - x_u)
            valid_u = (norm_int <= self.p//self.lambd)
        return u

    def define_target(self, s):
        self.targ = [self.lambd * s * x_i for x_i in self.x]
        self.targ.append(0)
        
    def solve_exact(self):
        cv_exact = self.lat.closest_vector(tuple(self.targ))
        return (-cv_exact)*self.G
        

    def solve_approx(self):
        l_b = matrix(self.lat.reduced_basis)
        t_b = matrix(1, self.l+1, [-t_i for t_i in self.targ])
        z_b = matrix(self.l+1, 1, [0]*(self.l+1))
        M_b = matrix(1, 1, [self.p ** 2])
        l1 = block_matrix(2, 2, [ [l_b, z_b], [t_b, M_b]])
        l1_reduced = IntegerLattice(l1).reduced_basis
        delta = np.array(l1_reduced[self.l+1][:-1])
        y1 = -1 * matrix(list(self.targ + delta))
        return y1*self.G
        

In [610]:
bv = BV(7, 2, 2)
exact = bv.solve_exact()
approx = bv.solve_approx()
print(exact)
print(approx)

(0, 0, 0)
[    0 -3136     0]
